#### 개정 일자: 2020.11.02


#### 2020.11.02

1) 뉴스검색결과 전체 건수 항목 사라져서 삭제

#### 2020.05.02
1) 시작날짜, 종료날짜 입력하여 실행하도록 설정

2) column단위 병합에서 row 단위 개별 추가로 변경

In [1]:
import re
import time
import requests
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [2]:
#수집 : keyword(키워드, sortNo(정렬기준)
def searchNewsExtractor1(keyword,sortNo): #수동
    global soup
    #pageNcnt = soup.find('div', {'class' : 'title_desc all_my'}).text
    #match = re.search('\d*,*\d*건',pageNcnt)
    news_count = int(str(match.group())[:-1].replace(",",""))
    
    for i in range(news_count//10+1): #4,000건이 최대    
        pageNo= 10*i+1
        url = "https://search.naver.com/search.naver?&where=news&query="+keyword+"&sm=tab_pge&sort="+str(sortNo)+"&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:da,p:all,a:all&mynews=0&start="+str(pageNo)+"&refresh_start=0"
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'lxml')
        newsParser()
        
        if req.status_code==200:
            print("뉴스 ",pageNo,"번째 수집 완료")

In [3]:
#수집 : keyword/ 정렬순서(최신/과거)/ 날짜 지정 
def searchNewsExtractor2(keyword,sortNo,sdate,edate): #자동
    global soup
    dot_sdate = sdate[0:4]+"."+sdate[4:6]+"."+sdate[6:8]
    dot_edate = edate[0:4]+"."+edate[4:6]+"."+edate[6:8]
    
    url1 = "https://search.naver.com/search.naver?&where=news&query="+keyword+"&sm=tab_pge&sort="+str(sortNo)+"&photo=0&field=0&reporter_article=&pd=3&ds="+dot_sdate+"&de="+dot_edate+"&docid=&nso=so:dd,p:from"+sdate+"to"+edate+"%2Ca%3Aall&mynews=0&start=1+&refresh_start=0"
    req1 = requests.get(url1)
    html1 = req1.text
    soup1 = BeautifulSoup(html1, 'lxml')
    
    pageNcnt = soup1.find('div', {'class' : 'title_desc all_my'}).text
    match = re.search('\d*,*\d*건',pageNcnt)
    news_count = int(str(match.group())[:-1].replace(",",""))
    
    for i in range(news_count//10+1): #4,000건이 최대
        pageNo= 10*i+1
        url = "https://search.naver.com/search.naver?&where=news&query="+keyword+"&sm=tab_pge&sort="+str(sortNo)+"&photo=0&field=0&reporter_article=&pd=3&ds="+dot_sdate+"&de="+dot_edate+"&docid=&nso=so:dd,p:from"+sdate+"to"+edate+"%2Ca%3Aall&mynews=0&start="+str(pageNo)+"&refresh_start=0"
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'lxml')
        newsParser()
        
        if req.status_code==200:
            print(keyword,"/",sdate,"~",edate,"--> " ,pageNo,"/",news_count)
            #print(url)

In [4]:
def newsParser():
    global href_list
    global title_list
    global press_list
    global date_list
    global content_list

    #1. 링크, 2. 타이틀
    hrefNtitle = soup.find_all(class_=[" _sp_each_title","_sp_each_url _sp_each_title"])    
    for i in hrefNtitle:
        href_list.append(i['href']) 
        title_list.append(i['title'])
        
    #3. 언론사
    press = soup.find_all("span",{"class":"_sp_each_source"})
    for i in press:
        press_list.append(i.text)

    #4. 날짜(3가지 유형 - 날짜/일 전/ 시간 전)
    sourceNdate= soup.find_all("dd",{"class":"txt_inline"})

    for i in sourceNdate:
        try: 
            match = re.search(r'\d{4}.\d{2}.\d{2}', i.text)
            date_list.append(str(match.group()))
        except AttributeError:
            try: 
                if "일" in i.text:
                    match = re.search('\d+일 전', i.text)
                    date_list.append(str(match.group())) 
                elif "시간" in i.text:
                    match = re.search('\d+시간 전', i.text)
                    date_list.append(str(match.group()))
                else: pass
            except AttributeError: 
                date_list.append(i.text)
            
    # 5. 본문
    articles = soup.select('.type01 > li')
    for i in range(len(articles)):
        content_list.append(articles[i].select('dl > dd')[1].text)

In [5]:
def to_excel(filenm):
    global df1   
    df1 =  DataFrame({"date":date_list,
                      "press": press_list, 
                      "title": title_list,  
                      "content": content_list,
                      "url":href_list})
    
    for i in range(len(df1)):
        #print(df1.date[i])

        #최신날짜 "-일전/-시간 전" 정제하기 
        if "일" in df1.date[i]:
            match = re.search('\d+', df1.date[i])
            now = datetime.now() #현재 시간(수집 시간)
            diff = int(match.group()) #몇일 전/몇시간 전인지
            date = now - timedelta(days=diff)
            df1.date[i] = date.strftime("%Y.%m.%d")

        elif "시간" in df1.date[i]:
            match = re.search('\d+', df1.date[i])
            now = datetime.now() #현재 시간(수집 시간)
            diff = int(match.group()) #몇일 전/몇시간 전인지
            date = now - timedelta(hours=diff)
            
            df1.date[i] = date.strftime("%Y.%m.%d")
            #print("몇시간전? :",diff,"\n현재시각: ",now, "\n바뀐 시간: ", date)
            #print("입력시간: ",date.strftime("%Y.%m.%d"))
        else: pass
    
    df1.to_excel(filenm, index=True)
    return df1

In [6]:
#####-------------main-----------------#####
href_list = []
title_list = []
press_list = []
date_list = []
content_list = []

keyword = "카운터포인트리서치" #카운터포인트리서치/스트래티지애널리틱스
sortNo = 1 #1:최신순, 2: 과거순
sdate = "20201001" #시작 날짜
edate = "20201030" #종료 날짜 
filenm = keyword+"_"+sdate+"_"+edate+".xlsx" #저장할 파일명

#1. 최신순/과거순
#searchNewsExtractor1(keyword,sortNo)

#2. 날짜지정
searchNewsExtractor2(keyword,sortNo,sdate,edate)
to_excel(filenm)

AttributeError: 'NoneType' object has no attribute 'text'